In [0]:
%pip install -r requirements.txt

In [0]:
import pyspark.sql.functions as f
import sys

import entry_point
import importlib
import dbks_dbms.mgr.dbks_dms_mgr
import dbks_dbms.stg_tbl_mgr.mysql_tbl_mgr
import dbks_dbms.mysql_rw.mysql_writer
from dbks_dbms.stg_tbl_mgr.mysql_tbl_mgr import MysqlTblMgr
from dbks_dbms.mysql_rw.mysql_writer import MysqlWriter

In [0]:
src_table_name="dw_dim_state"
src_database="Volumes/ptudfpii/pii/conformedpiiedw/dw"
tgt_db_typ="mysql"


In [0]:
def get_current_notebook_path():
    notebook_path = dbutils.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    return notebook_path

def get_environment_from_notebook_path():
    #note_book_path = get_current_notebook_path()
    #main_logger.info(f"Notebook path :{note_book_path}")
    if  spark.conf.get("spark.databricks.clusterUsageTags.clusterOwnerOrgId")=='4022166418081681':
        return 'PT'
    elif spark.conf.get("spark.databricks.clusterUsageTags.clusterOwnerOrgId")=='add production id':
        return 'prod'

def get_user_mysql_cred(mysql_env,secret_mgr_scope='ds-edw-prod'):
    if mysql_env.lower() in ['pt','prod']:
        mysql_ip='edwawsdatasci01.aws2.teladoc.com' #dbutils.secrets.get(scope=secret_mgr_scope, key='HOST_NAME')
        mysql_port=3306 #dbutils.secrets.get(scope=secret_mgr_scope, key='HOST_PORT')
        musql_username='databricks' #dbutils.secrets.get(scope=secret_mgr_scope, key='USERNAME')
        mysql_password='masonry_e3' #dbutils.secrets.get(scope=secret_mgr_scope, key='PASSWORD')
        return (mysql_ip,mysql_port,musql_username,mysql_password)
    else:
        raise Exception(f"Invalid environment for mysql user credentials {mysql_env}")

def get_redshift_user_cred(redshift_env,secret_mgr_scope):
    if redshift_env.lower() in ['prod']:
        redshift_ip_user = None #dbutils.secrets.get(scope=secret_mgr_scope, key="redshift_talend_username")
        redshift_password = None #dbutils.secrets.get(scope=secret_mgr_scope, key="redshift_talend_password")
        return redshift_ip_user,redshift_password
    else:
        raise Exception(f"Invalid environment for redshift user credentials {mysql_env}")


In [0]:


importlib.reload(entry_point)
environment = get_environment_from_notebook_path()




In [0]:
 
pathlist = ['/Workspace/Users/madan.hajare@teladochealth.com/dbks_dbms','/Workspace/Users/madan.hajare@teladochealth.com/dbks_dbms/mgr','/Workspace/Users/madan.hajare@teladochealth.com/dbks_dbms/mgr/dbks_dms_mgr','/Workspace/Users/madan.hajare@teladochealth.com/dbks_dbms/stg_tbl_mgr/mysql_tbl_mgr'
            ,'dbks_dbms/mysql_rw/mysql_writer']

for path in pathlist:
    if path not in sys.path:
        print("Adding path: " + path)
        sys.path.append(path)


importlib.reload(dbks_dbms.mgr.dbks_dms_mgr)
importlib.reload(dbks_dbms.stg_tbl_mgr.mysql_tbl_mgr)
importlib.reload(dbks_dbms.mysql_rw.mysql_writer)

In [0]:

df=spark.table("udf_internal.udf_to_rdbms_sync").where(f"src_tbl='{src_table_name}'").select("udf_to_rdbms_sync_key","src_db","src_tbl","tgt_db_typ","tgt_db","tgt_tbl","merge_keys")

assert df.count() > 0, "No records found in udf_internal.udf_to_rdbms_sync"

# check last successful log entry and get watermark value


for row in df.collect():
    mysql_ip,mysql_port,mysql_username,mysql_password = get_user_mysql_cred(mysql_env=environment,secret_mgr_scope='ds-edw-prod')
    redshift_user,redshift_pass = get_redshift_user_cred(redshift_env='prod',secret_mgr_scope='ds-edw-prod')

 

    sql_conn= (mysql_ip,mysql_port,mysql_username,mysql_password,redshift_user,redshift_pass)    
    master_obj= entry_point.start_dbks_dms(row,sql_conn)
  
                                    




    